In [ ]:
pip install autokeras -q

     |████████████████████████████████| 174kB 6.0MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 


### Tune the number of units of a two-layer MLP

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
  input_node = keras.Input(shape=(20,))
  units = hp.Int('units', min_value=32, max_value=512, step=32)
  output_node = layers.Dense(units=units, activation='relu')(input_node)
  output_node = layers.Dense(units=units, activation='relu')(output_node)
  output_node = layers.Dense(units=1, activation='sigmoid')(output_node)
  model = keras.Model(input_node, output_node)
 
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
  model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mae'])
  return model

In [ ]:
from kerastuner import RandomSearch

tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [ ]:
import numpy as np

x_train = np.random.rand(100, 20)
y_train = np.random.rand(100, 1)
x_val = np.random.rand(20, 20)
y_val = np.random.rand(20, 1)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_val, y_val))
tuner.results_summary(5)

Trial 5 Complete [00h 00m 02s]
val_mae: 0.2220905969540278

Best val_mae So Far: 0.2120091120402018
Total elapsed time: 00h 00m 11s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_mae', direction='min')
Trial summary
Hyperparameters:
units: 288
Score: 0.2120091120402018
Trial summary
Hyperparameters:
units: 128
Score: 0.2220905969540278
Trial summary
Hyperparameters:
units: 320
Score: 0.22237977385520935
Trial summary
Hyperparameters:
units: 256
Score: 0.22893168032169342
Trial summary
Hyperparameters:
units: 192
Score: 0.23000877102216086


In [ ]:
best_models = tuner.get_best_models(num_models=2)
best_model = best_models[0]
best_model.save('path_to_best_model')
print(best_model.predict(x_val))
best_model.summary()

INFO:tensorflow:Assets written to: path_to_best_model/assets
[[0.4977915 ]
 [0.41781846]
 [0.50711876]
 [0.47860923]
 [0.41595194]
 [0.4969605 ]
 [0.44580874]
 [0.42493588]
 [0.4926258 ]
 [0.47569945]
 [0.4430859 ]
 [0.46065745]
 [0.497313  ]
 [0.48311725]
 [0.46327966]
 [0.46197027]
 [0.45616627]
 [0.46178862]
 [0.4308407 ]
 [0.46655235]]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
dense (Dense)                (None, 288)               6048      
_________________________________________________________________
dense_1 (Dense)              (None, 288)               83232     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 289       
Total params: 89,569
Trainable params: 89,569
Non

### Jointly tune the optimization function

In [ ]:
def rmse(y_true, y_pred):
  return tf.keras.backend.sqrt(tf.keras.losses.MSE(y_true, y_pred))

def build_model(hp):
  input_node = keras.Input(shape=(20,))
  units = hp.Int('units', min_value=32, max_value=512, step=32)
  output_node = layers.Dense(units=units, activation='relu')(input_node)
  output_node = layers.Dense(units=units, activation='relu')(output_node)
  output_node = layers.Dense(units=1, activation='sigmoid')(output_node)
  model = keras.Model(input_node, output_node)
  optimizer_name = hp.Choice('optimizer', ['adam', 'adadelta'])
  learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=0.1, sampling='log')
  if optimizer_name == 'adam':
    optimizer = tf.keras.optimizer.Adam(learning_rate=learning_rate)
  else:
    optimizer = tf.keras.optimizer.Adadelta(learning_rate=learning_rate)
  model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=['mae', rmse])
  return model

### Tune the data preprocessing method

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

layer = Normalization(input_shape=(20,))
layer.adapt(x_train)

model = tf.keras.Sequential([layer, tf.keras.layers.Dense(1)])
model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train)

4/4 [==============================] - 0s 3ms/step - loss: 1.2505


In [ ]:
from kerastuner import HyperModel

class Regressor(HyperModel):
  def __init__(self, data):
    self.data = data

  def build(self, hp):
    model = tf.keras.Sequential()
    if hp.Boolean('nomalize'):
        layer = Normalization(input_shape=(20,))
        layer.adapt(self.data)
        model.add(layer)
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model